In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determin

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY_1')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo' # You can change this to other GPT versions
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [5]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.38it/s]


In [7]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

In [8]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [9]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do incredible research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [10]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [11]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

In [12]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [13]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [14]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [15]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

In [16]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

In [17]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [18]:
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

In [19]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Summary
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, excelling in leading remote and in-office engineering teams. His expertise spans software development, process innovation, and enhancing team collaboration. Highly proficient in Ruby, Python, JavaScript, TypeScript, and Elixir, with deep expertise in various front end frameworks. Significant experience in data science and machine learning.

## Work Experience

### Director of Software Engineering
DataKernel | 2022 - Present
- Transformed the engineering division into a key revenue pillar, expanding customer base and integrating cutting-edge AI technologies.
- Led team growth and skill development, defining long-term strategic initiatives.

### Senior Software Engineering Manager
DataKernel | 2019 - 2022
- Directed engineering strategy and operations, managed diverse teams across time zones, and fostered a culture of continuous improvement.

### Founder & CEO
InnovPet | 2019 - 2022
- Founded startup focused on IoT solutions for pet care, overseeing product development and successful market entry.

### Engineering Manager
EliteDevs | 2018 - 2019
- Formulated and executed strategic plans, managed multiple teams, and implemented goal-setting frameworks.

### Engineering Manager
PrintPack | 2016 - 2018
- Led development of high-performance engineering team, integrating data analytics and developing predictive modeling tool.

### Senior Software Engineer
DriveAI | 2015 - 2016
- Developed and optimized central API, implemented critical enhancements, and improved system performance.

### CTO
BetCraft | 2013 - 2015
- Led technology strategy post-Series A funding, improved platform performance, and expanded market reach.

## Skills
- Proficient in Ruby, Python, JavaScript, TypeScript, and Elixir
- Expertise in front end frameworks
- Strong background in data science and machine learning
- Leadership and strategic planning
- Team management and collaboration
- Product development and market entry

## Education
- MBA in Information Technology, London Business School
- Advanced Leadership Techniques, University of London
- Data Science Specialization, Coursera (Johns Hopkins University)
- B.Sc. in Computer Science, University of Edinburgh

Noah Williams is an ideal candidate for senior executive roles, offering a robust blend of technical and strategic expertise.

In [20]:
display(Markdown("./interview_materials.md"))

## Potential Interview Questions and Talking Points for Noah Williams:

1. Can you walk us through your experience in leading remote and in-office engineering teams, and how you have managed to foster collaboration and teamwork effectively in both settings?
2. You have a strong background in AI and data science. Could you discuss a specific project where you implemented AI technologies or developed innovative data models that had a significant impact on the product or company?
3. Given your proficiency in multiple programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, can you share an example of a project where you leveraged these languages to solve a complex technical challenge?
4. As someone with expertise in software development frameworks like Rails API and Active Model Serializers, how do you approach selecting the right framework for a project, and what factors do you consider in your decision-making process?
5. You specialize in gamification and ranking algorithm implementation. Could you provide an example of a project where you incorporated gamification elements or ranking algorithms to enhance user engagement or product performance?
6. With your knowledge in authentication solutions for Phoenix, can you discuss a specific authentication challenge you faced in a project and how you overcame it using your expertise?
7. Tell us about your experience founding crewAIInc and the challenges you encountered in orchestrating role-playing autonomous AI agents. How did you overcome these challenges and drive the project to success?
8. Could you share a project where you developed machinery, an Elixir State machine thin layer for structs, and the benefits it brought to the development process or product functionality?
9. As a contributor to rails-api/active_model_serializers, can you discuss your contributions to the project and how they have impacted the Rails community or development practices?
10. How do you stay updated on the latest technologies and innovations in the tech industry, and how do you incorporate these advancements into your work and projects?

Talking Points:
- Highlight your experience in leading major tech initiatives and startups, showcasing your ability to drive innovation and growth in the tech industry.
- Emphasize your strong communication skills and collaborative approach to working with teams, demonstrating your ability to convey complex technical concepts clearly and foster collaborative intelligence.
- Discuss your leadership skills in actively managing and mentoring teams, setting strategic goals, and achieving successful project outcomes.
- Showcase your passion for AI, data science, and software development frameworks, along with your interest in gamification and ranking algorithms, as key drivers of your professional motivation and growth.
- Mention your educational background, including an MBA in Information Technology, advanced leadership techniques, and data science specialization, to underscore your commitment to continuous learning and professional development.